In [1]:
import pandas as pd
import xmlrpc.client
import glob

In [2]:
# Ruta donde están los archivos
ruta = "/home/donsson/proyectos/SIMON/exports/*.xlsx"

# Leer todos los excels en la carpeta
archivos = glob.glob(ruta)

dfs = []
for archivo in archivos:
    print(f"Leyendo: {archivo}")
    df_temp = pd.read_excel(archivo)
    dfs.append(df_temp)


pd.set_option("display.float_format", "{:,.0f}".format)

# Unir todos en un único DataFrame
df_total = pd.concat(dfs, ignore_index=True)

df_total['product_marca'] = df_total['product_marca'].fillna("Indefinido")

df_total["date"] = pd.to_datetime(df_total["date"])

df_total["año"] = df_total["date"].dt.year
df_total["mes"] = df_total["date"].dt.month

print("______________________________________________________________________")


print("Total de registros:", len(df_total))
df_total.info()

#Tarda 1 minuto

Leyendo: /home/donsson/proyectos/SIMON/exports/ventas_2024_20250923_201745.xlsx
Leyendo: /home/donsson/proyectos/SIMON/exports/ventas_2025_20250924_084803.xlsx
Leyendo: /home/donsson/proyectos/SIMON/exports/ventas_2022_20250923_183845.xlsx
Leyendo: /home/donsson/proyectos/SIMON/exports/ventas_2023_20250923_192209.xlsx
Leyendo: /home/donsson/proyectos/SIMON/exports/ventas_2020_20250923_171735.xlsx
Leyendo: /home/donsson/proyectos/SIMON/exports/ventas_2021_20250923_180002.xlsx
______________________________________________________________________
Total de registros: 893003
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893003 entries, 0 to 893002
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           893003 non-null  datetime64[ns]
 1   price_total    893003 non-null  float64       
 2   product_cost   893003 non-null  float64       
 3   id             893003 non-null  int64         


In [12]:
mapeo = {
    "DONSSON": "DONSSON",
    "BALDWIN": "BALDWIN",
    "RACOR BRASIL": "PARKER",
    "RACOR USA": "PARKER",
    "DELO": "ACEITE",
    "MOBIL IMPORTADO": "ACEITE",
    "MOBIL NACIONAL": "ACEITE",
    "KOMATSU": "ACEITE",
    "PURE GUARD": "ACEITE",
    "URSA": "ACEITE",
    "SUPREME": "ACEITE",
    "HIDROCOOL": "REFRIGERANTES",
    "TEKCOOL": "REFRIGERANTES",
    "OTROS": "OTROS",
    "False": "OTROS",
    "Indefinido": "OTROS",
    "RAMA": "OTROS_FILTROS",
    "PERKINS": "OTROS_FILTROS",
    "AUT*PARTS": "AUT*PARTS"
}

df_total["marca_agrupada"] = df_total["product_marca"].map(mapeo).fillna("OTROS")


In [13]:
import pandas as pd

# Crear la tabla pivote
tabla = pd.pivot_table(
    df_total,
    index="marca_agrupada",
    columns="año",
    values={"price_total": "sum", "product_cost": "sum"},
    aggfunc="sum",
    fill_value=0,
    margins=True,
    margins_name="TOTAL"
)

# Renombrar las métricas con nombres más claros
tabla = tabla.rename(
    columns={
        "price_total": "Total Without Tax",
        "product_cost": "Total costo"
    }
)

# Reordenar niveles de columnas: primero año, luego métrica
tabla = tabla.reorder_levels([1, 0], axis=1).sort_index(axis=1, level=0)

tabla.to_excel("infrome_marcas_account_invoice.xlsx")

tabla.head(20)



año                         2020                            2021  \
               Total Without Tax   Total costo Total Without Tax   
marca_agrupada                                                     
ACEITE               404,424,300   319,032,155       555,894,499   
AUT*PARTS            254,016,686   147,441,774       570,087,642   
BALDWIN            6,811,217,170 4,628,748,312     8,943,498,872   
DONSSON            6,284,684,012 2,912,889,224     9,517,930,917   
OTROS                323,871,483   156,350,386       290,071,095   
OTROS_FILTROS          1,583,524       904,512         2,191,500   
PARKER             1,307,766,045   804,839,990     1,598,938,402   
REFRIGERANTES          5,204,425     3,128,618         7,160,900   
TOTAL             15,392,767,645 8,973,334,971    21,485,773,828   

año                                        2022                 \
                  Total costo Total Without Tax    Total costo   
marca_agrupada                                                   
ACEITE            435,382,054       681,412,900    497,011,617   
AUT*PARTS         326,666,582       916,944,486    499,604,286   
BALDWIN         5,946,091,302    10,379,533,330  6,587,575,303   
DONSSON         4,583,238,471    12,955,927,227  6,227,651,678   
OTROS             154,202,937       412,407,266    164,967,689   
OTROS_FILTROS         910,572         6,399,667      2,994,515   
PARKER            981,805,070     2,018,813,557  1,252,881,998   
REFRIGERANTES       4,355,172         6,502,900      3,934,898   
TOTAL          12,432,652,159    27,377,941,333 15,236,621,985   

año                         2023                             2024  \
               Total Without Tax    Total costo Total Without Tax   
marca_agrupada                                                      
ACEITE               755,631,900    554,980,229       774,324,900   
AUT*PARTS          1,330,927,800    704,229,026     1,575,114,265   
BALDWIN           12,568,365,444  8,118,719,444    13,505,865,262   
DONSSON           15,301,121,870  6,886,257,882    14,845,977,802   
OTROS                 96,948,583    105,148,945       177,715,960   
OTROS_FILTROS          2,687,500      1,227,907         2,770,500   
PARKER             1,924,385,072  1,164,013,505     1,768,850,214   
REFRIGERANTES          6,638,200      4,141,724        11,425,700   
TOTAL             31,986,706,369 17,538,718,662    32,662,044,602   

año                                        2025                 \
                  Total costo Total Without Tax    Total costo   
marca_agrupada                                                   
ACEITE            575,984,627       575,530,600    424,262,070   
AUT*PARTS         733,147,827     1,468,209,300    649,805,519   
BALDWIN         9,563,852,950    11,232,321,760  7,954,065,438   
DONSSON         6,451,901,737    11,409,234,961  5,130,283,452   
OTROS              94,305,585       184,490,280     76,775,169   
OTROS_FILTROS         147,406         1,485,800        681,443   
PARKER          1,113,540,328     1,561,093,024    991,180,650   
REFRIGERANTES       7,011,470        24,979,900     15,548,989   
TOTAL          18,539,891,929    26,457,345,625 15,242,602,731   

año                        TOTAL                 
               Total Without Tax    Total costo  
marca_agrupada                                   
ACEITE             3,747,219,099  2,806,652,753  
AUT*PARTS          6,115,300,179  3,060,895,013  
BALDWIN           63,440,801,838 42,799,052,748  
DONSSON           70,314,876,790 32,192,222,445  
OTROS              1,485,504,667    751,750,710  
OTROS_FILTROS         17,118,491      6,866,355  
PARKER            10,179,846,314  6,308,261,542  
REFRIGERANTES         61,912,025     38,120,871  
TOTAL            155,362,579,402 87,963,822,436